In [1]:
%cd ..

c:\Users\HP\OneDrive - University of Moratuwa\Desktop\TMS-Project\TMS-Backend


#### Update matrix without database

In [2]:
import pandas as pd 
from src.database.database import engine
from config.settings import Settings
from src.utils.master_utils import get_osrm_data
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

class MasterManager:
    def __init__(self, configs=None):
        self.config = configs
        if self.config is None:
            self.config = Settings()
            
        self.gps_df = self._load_gps()
        
        self.distance_matrix_path = self.config.distance_matrix_file_path
        self.duration_matrix_path = self.config.duration_matrix_file_path
        
    def _load_gps(self):
        df = pd.read_sql("SELECT * FROM master_gps", engine)
        gps_csv_path = self.config.master_folder_path / self.config.gps_file_name
        df.to_csv(gps_csv_path, index=False)
        return df
    
    # def update_matrix(self):
    #     start = time.time()
    #     if not self.distance_matrix_path.exists():
    #         # gps locations and its shop codes
    #         shop_codes = self.gps_df['shop_code'].values
    #         coords = self.gps_df[['latitude', 'longitude']].values
    #         # create  matrixes
    #         distance_matrix = np.zeros((len(shop_codes), len(shop_codes)))
    #         duration_matrix = np.zeros((len(shop_codes), len(shop_codes)))
    #         for i in range(len(shop_codes)):
    #             for j  in range(i, len(shop_codes)):
    #                 if i == j:
    #                     distance_matrix[i][j] = 0
    #                     duration_matrix[i][j] = 0
    #                     continue
    #                 origin = tuple(coords[i])
    #                 destination = tuple(coords[j])
                
    #                 _, distance, duration = get_osrm_data(origin, destination)
                    
    #                 distance_matrix[i][j] = round(distance, 3)
    #                 distance_matrix[j][i] = round(distance, 3)
                    
    #                 duration_matrix[i][j] = round(duration, 2)
    #                 duration_matrix[j][i] = round(duration, 2)
    #     print(time.time()-start)
    #     print(distance_matrix)
    #     print(duration_matrix)

    def _get_osrm_thread(self, origin, destination):
        _, distance, duration = get_osrm_data(origin, destination)
        return round(distance, 3), round(duration, 2)

    def update_matrix(self):
        start = time.time()
        if not self.distance_matrix_path.exists():
            # gps locations and its shop codes
            shop_codes = self.gps_df['shop_code'].values
            coords = self.gps_df[['latitude', 'longitude']].values
            # create matrixes
            distance_matrix = np.zeros((len(shop_codes), len(shop_codes)))
            duration_matrix = np.zeros((len(shop_codes), len(shop_codes)))
            
            # **THREAD MAGIC: REPLACE YOUR LOOP WITH THESE 6 LINES**
            with ThreadPoolExecutor(max_workers=50) as executor:
                futures = {}
                for i in range(len(shop_codes)):
                    for j in range(i+1, len(shop_codes)):  # Skip i==j & duplicates
                        origin = tuple(coords[i])
                        destination = tuple(coords[j])
                        future = executor.submit(self._get_osrm_thread, origin, destination)
                        futures[future] = (i, j)
                
                for future in as_completed(futures):
                    i, j = futures[future]
                    distance, duration = future.result()
                    distance_matrix[i][j] = distance
                    distance_matrix[j][i] = distance
                    duration_matrix[i][j] = duration
                    duration_matrix[j][i] = duration
                    
            distance_df = pd.DataFrame(distance_matrix, columns=shop_codes, index=list(shop_codes))
            duration_df = pd.DataFrame(duration_matrix, columns=shop_codes, index=list(shop_codes))
            
            distance_df.to_csv(self.distance_matrix_path)
            duration_df.to_csv(self.duration_matrix_path)
            self.gps_df.to_csv(self.config.master_folder_path/'previous_gps.csv', index=False)
        print(time.time() - start)     
        print(distance_matrix)
        print(duration_matrix)

In [16]:
manager = MasterManager()

In [17]:
list(manager.gps_df['shop_code'].values)

['0', '3', '4', '5']

In [18]:
manager.update_matrix()

20.282389640808105
[[ 0.     7.41  24.93  16.722]
 [ 7.41   0.    18.816 10.303]
 [24.93  18.816  0.    11.478]
 [16.722 10.303 11.478  0.   ]]
[[ 0.    9.13 32.39 21.23]
 [ 9.13  0.   25.71 14.04]
 [32.39 25.71  0.   15.92]
 [21.23 14.04 15.92  0.  ]]


In [119]:
manager.update_matrix()

6.137975692749023
[[ 0.     7.41  24.93  16.722]
 [ 7.41   0.    18.816 10.303]
 [24.93  18.816  0.    11.478]
 [16.722 10.303 11.478  0.   ]]
[[ 0.    9.13 32.39 21.23]
 [ 9.13  0.   25.71 14.04]
 [32.39 25.71  0.   15.92]
 [21.23 14.04 15.92  0.  ]]


#### Get distance and time without recalculating with database update

In [ ]:
from sqlalchemy.orm import Session
from typing import List, Dict, Tuple
from datetime import datetime
from src.utils.master_utils import get_osrm_data
from src.database.models import GPSMaster, MatrixMaster

class DistanceMatrixManager:
    """Manager class for distance matrix operations"""
    
    def __init__(self, db_session: Session):
        self.db = db_session
        
    def calculate_distance(self, lat1: float, lon1: float, 
                          lat2: float, lon2: float) -> float:
        """
        Calculate Haversine distance between two GPS coordinates.
        Returns distance in kilometers.
        """
        _, distance, duration = get_osrm_data((lat1,lon1), (lat2,lon2))
        return distance, duration
    
    def process_pending_updates(self):
        """
        Process all shops marked with matrix_status = 'to_update'.
        Calculate distances and update matrix_status to 'updated'.
        """
        # Get shops that need matrix update
        pending_shops = self.db.query(GPSMaster).filter(
            (GPSMaster.matrix_status == 'to_update') | 
            (GPSMaster.matrix_status == 'to_create')
        ).all()
        
        if not pending_shops:
            print("No pending updates found.")
            return 0
                # Get all shops that are already in the matrix (status = 'updated')
        existing_shops = self.db.query(GPSMaster).filter(
            GPSMaster.matrix_status == 'updated'
        ).all()
        
        total_calculations = 0
        for pending_shop in pending_shops:
            print(f"Processing shop: {pending_shop.shop_code} (ID: {pending_shop.id})")
            
            # Delete old distances if this shop was previously in matrix
            self._delete_shop_distances(pending_shop.id)
            
            # Calculate distances to all existing shops
            distances_added = self._add_shop_to_matrix(
                pending_shop, 
                existing_shops
            )
            
            # Also calculate distances to other pending shops
            for other_pending in pending_shops:
                if other_pending.id != pending_shop.id:
                    self._add_distance_pair(pending_shop, other_pending)
                    distances_added += 1
            
            # Update status to 'updated'
            pending_shop.matrix_status = 'updated'
            total_calculations += distances_added
            

        self.db.commit()
        print(f"\nTotal: Processed {len(pending_shops)} shops, "
              f"{total_calculations} distance calculations")
        
        return total_calculations
    
    def _delete_shop_distances(self, shop_id: int):
        """Delete all distance entries for a specific shop"""
        self.db.query(MatrixMaster).filter(
            (MatrixMaster.shop_id_1 == shop_id) |
            (MatrixMaster.shop_id_2 == shop_id)
        ).delete(synchronize_session=False)
        
        
    def _add_shop_to_matrix(self, new_shop: GPSMaster, 
                           existing_shops: List[GPSMaster]) -> int:
        """
        Add a new shop to the matrix by calculating distances to all existing shops.
        Returns count of distances added.
        """
        
        count = 0
        for existing_shop in existing_shops:
            if existing_shop.id != new_shop.id:
                self._add_distance_pair(new_shop, existing_shop)
                count += 1
        
        return count
    
    def _add_distance_pair(self, shop1: GPSMaster, shop2: GPSMaster):
        """
        Add or update a single distance pair to the matrix.
        Ensures shop_id_1 < shop_id_2 for upper triangle storage.
        """
        distance, time = self.calculate_distance(
            shop1.latitude, shop1.longitude,
            shop2.latitude, shop2.longitude
        )
        
        # Check if distance already exists
        existing = self.db.query(MatrixMaster).filter(
            MatrixMaster.shop_id_1 == shop1.id,
            MatrixMaster.shop_id_2 == shop2.id
        ).first()
        
        if existing:
            existing.distance_km = distance
            existing.time_minutes = time
            existing.last_calculated = datetime.utcnow()
        else:
            new_distance = MatrixMaster(
                shop_id_1=shop1.id,
                shop_id_2=shop2.id,
                shop_code_1=shop1.shop_code,
                shop_code_2=shop2.shop_code,
                distance_km=distance,
                time_minutes=time,
                last_calculated=datetime.utcnow()
            )
            self.db.add(new_distance)

In [7]:
from src.database.database import SessionLocal

manager = DistanceMatrixManager(SessionLocal())

In [8]:
x = manager.process_pending_updates()

Processing shop: 0 (ID: 1)
Processing shop: 3 (ID: 2)
Processing shop: 4 (ID: 3)
Processing shop: 5 (ID: 4)

Total: Processed 4 shops, 12 distance calculations


#### Get matrix with threading and without recalculating with updating database

In [5]:
from sqlalchemy.orm import Session
from typing import List, Dict, Tuple
from datetime import datetime
from src.utils.master_utils import get_osrm_data
from src.database.models import GPSMaster, MatrixMaster
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

In [40]:
class DistanceMatrixManager:
    """Fixed: Thread-safe distance matrix manager"""
    
    def __init__(self, db_session: Session, max_workers: int = 5):  # Reduced workers
        self.db = db_session
        self.max_workers = max_workers
        
    def calculate_distance(self, lat1: float, lon1: float, 
                          lat2: float, lon2: float) -> Tuple[float, float]:
        """
        Calculate distance and time between two GPS coordinates using OSRM.
        Returns (distance_km, duration_minutes).
        """
        _, distance, duration = get_osrm_data((lat1, lon1), (lat2, lon2))
        return distance, duration
    def process_pending_updates(self):
        """FIXED: Proper commit order + thread safety"""
        pending_shops = self.db.query(GPSMaster).filter(
            (GPSMaster.matrix_status == 'to_update') | 
            (GPSMaster.matrix_status == 'to_create')
        ).all()
        
        if not pending_shops:
            print("No pending updates found.")
            return 0
        
        existing_shops = self.db.query(GPSMaster).filter(
            GPSMaster.matrix_status == 'updated'
        ).all()
        
        print(f"Processing {len(pending_shops)} pending shops...")
        total_calculations = 0
        
        # FIX #1: Process ONE SHOP AT A TIME (no threading conflicts)
        for pending_shop in pending_shops:
            print(f"\n{'='*60}")
            print(f"Processing shop: {pending_shop.shop_code} (ID: {pending_shop.id})")
            
            # Delete old distances
            self._delete_shop_distances(pending_shop.id)
            
            # Calculate ALL distances (to existing + other pending)
            all_targets = existing_shops + [s for s in pending_shops if s.id != pending_shop.id]
            distance_results = self._calculate_distances_serial(pending_shop, all_targets)  # SERIAL!
            
            # FIX #2: SAVE + COMMIT in MAIN THREAD
            for result in distance_results:
                if result:  # Skip None results
                    self._save_distance_to_db(result)
            
            # COMMIT ONCE after ALL saves
            self.db.commit()
            total_calculations += len([r for r in distance_results if r])
            
            # Update status
            pending_shop.matrix_status = 'updated'
            
            print(f"✓ Saved {len([r for r in distance_results if r])} distances")
        
        # Final status commit
        self.db.commit()
        print(f"\n✓ COMPLETE: {total_calculations} total distances")
        return total_calculations
    
    def _calculate_distances_serial(self, source_shop: GPSMaster, target_shops: List[GPSMaster]) -> List[Dict]:
        """FIXED: Serial calculation (no threading issues)"""
        results = []
        for target_shop in target_shops:
            result = self._calculate_single_distance(source_shop, target_shop)
            if result:
                results.append(result)
        return results
    
    def _delete_shop_distances(self, shop_id: int):
        """Delete all distance entries for a shop"""
        self.db.query(MatrixMaster).filter(
            (MatrixMaster.shop_id_1 == shop_id) | (MatrixMaster.shop_id_2 == shop_id)
        ).delete(synchronize_session=False)
        # NO COMMIT here - batch with others
    
    def _save_distance_to_db(self, distance_data: Dict):
        """FIXED: Always in main thread, no threading"""
        sid1 = min(distance_data['shop_id_1'], distance_data['shop_id_2'])
        sid2 = max(distance_data['shop_id_1'], distance_data['shop_id_2'])
        
        if distance_data['shop_id_1'] == sid1:
            scode1, scode2 = distance_data['shop_code_1'], distance_data['shop_code_2']
        else:
            scode1, scode2 = distance_data['shop_code_2'], distance_data['shop_code_1']
        
        # Check existing
        existing = self.db.query(MatrixMaster).filter(
            MatrixMaster.shop_id_1 == sid1, MatrixMaster.shop_id_2 == sid2
        ).first()
        
        if existing:
            existing.distance_km = distance_data['distance_km']
            existing.time_minutes = distance_data['time_minutes']
            existing.last_calculated = datetime.utcnow()
        else:
            new_distance = MatrixMaster(
                shop_id_1=sid1, shop_id_2=sid2,
                shop_code_1=scode1, shop_code_2=scode2,
                distance_km=distance_data['distance_km'],
                time_minutes=distance_data['time_minutes'],
                last_calculated=datetime.utcnow()
            )
            self.db.add(new_distance)
    
    def _calculate_single_distance(self, shop1: GPSMaster, shop2: GPSMaster) -> Dict:
        """Unchanged - works fine"""
        try:
            distance, time = self.calculate_distance(
                shop1.latitude, shop1.longitude,
                shop2.latitude, shop2.longitude
            )
            return {
                'shop_id_1': shop1.id, 'shop_id_2': shop2.id,
                'shop_code_1': shop1.shop_code, 'shop_code_2': shop2.shop_code,
                'distance_km': distance, 'time_minutes': time
            }
        except Exception as e:
            print(f"✗ Error {shop1.shop_code} → {shop2.shop_code}: {e}")
            return None
    
    def get_distance(self, shop_id_1: int, shop_id_2: int) -> Tuple[float, float]:
        """FIXED: Handle None properly"""
        if shop_id_1 == shop_id_2:
            return 0.0, 0.0
        
        sid1, sid2 = min(shop_id_1, shop_id_2), max(shop_id_1, shop_id_2)
        result = self.db.query(MatrixMaster).filter(
            MatrixMaster.shop_id_1 == sid1, MatrixMaster.shop_id_2 == sid2
        ).first()
        
        if result:
            return result.distance_km or 0.0, result.time_minutes or 0.0
        return 0.0, 0.0  # FIX: Return 0 instead of None
    
    # Keep all other methods unchanged...
    def get_distance_matrix_for_shops(self, shop_ids: List[int]) -> Dict[Tuple[int, int], Dict]:
        n = len(shop_ids)
        distance_dict = {}
        for shop_id in shop_ids:
            distance_dict[(shop_id, shop_id)] = {'distance': 0.0, 'time': 0.0}
        
        for i, shop_id_1 in enumerate(shop_ids):
            for shop_id_2 in shop_ids[i+1:]:
                distance, time = self.get_distance(shop_id_1, shop_id_2)
                distance_dict[(shop_id_1, shop_id_2)] = {'distance': distance, 'time': time}
                distance_dict[(shop_id_2, shop_id_1)] = {'distance': distance, 'time': time}
        return distance_dict
    
    def get_distance_matrix_as_array(self, shop_ids: List[int]) -> List[List[float]]:
        n = len(shop_ids)
        matrix = [[0.0 for _ in range(n)] for _ in range(n)]
        distance_dict = self.get_distance_matrix_for_shops(shop_ids)
        for i, shop_id_1 in enumerate(shop_ids):
            for j, shop_id_2 in enumerate(shop_ids):
                if (shop_id_1, shop_id_2) in distance_dict:
                    matrix[i][j] = distance_dict[(shop_id_1, shop_id_2)]['distance']
        return matrix
    
    def get_time_matrix_as_array(self, shop_ids: List[int]) -> List[List[float]]:
        n = len(shop_ids)
        matrix = [[0.0 for _ in range(n)] for _ in range(n)]
        distance_dict = self.get_distance_matrix_for_shops(shop_ids)
        for i, shop_id_1 in enumerate(shop_ids):
            for j, shop_id_2 in enumerate(shop_ids):
                if (shop_id_1, shop_id_2) in distance_dict:
                    matrix[i][j] = distance_dict[(shop_id_1, shop_id_2)]['time']
        return matrix
    
    def get_matrix_statistics(self) -> Dict:
        total_shops = self.db.query(GPSMaster).filter(GPSMaster.matrix_status == 'updated').count()
        pending_shops = self.db.query(GPSMaster).filter(
            (GPSMaster.matrix_status == 'to_update') | (GPSMaster.matrix_status == 'to_create')
        ).count()
        total_distances = self.db.query(MatrixMaster).count()
        expected_distances = (total_shops * (total_shops - 1)) / 2 if total_shops > 0 else 0
        
        return {
            'total_shops_in_matrix': total_shops,
            'pending_updates': pending_shops,
            'total_distances_stored': total_distances,
            'expected_distances': int(expected_distances),
            'matrix_completeness': f"{(total_distances/expected_distances*100):.1f}%" if expected_distances > 0 else "N/A"
        }

In [41]:
# FIXED Usage
def example_usage():
    from src.database.database import SessionLocal
    
    db = SessionLocal()
    manager = DistanceMatrixManager(db, max_workers=5)
    
    try:
        print("=== FIXED Processing ===")
        manager.process_pending_updates()
        
        print("\n=== FIXED Statistics ===")
        stats = manager.get_matrix_statistics()
        for k, v in stats.items():
            print(f"{k}: {v}")
        
        shop_ids = [2, 3, 1, 5]  # FIXED: Use correct IDs
        distance_matrix = manager.get_distance_matrix_as_array(shop_ids)
        print(f"\nDistance Matrix:")
        for row in distance_matrix:
            print([f"{x:.2f}" for x in row])
        
        # FIXED: No more NoneType error
        dist, time = manager.get_distance(1, 2)
        print(f"Shop 1 → 2: {dist:.2f} km, {time:.2f} min")
        
    finally:
        db.commit() 
        db.close()

In [42]:
example_usage()

=== FIXED Processing ===
Processing 1 pending shops...

Processing shop: 105 (ID: 6)
✓ Saved 5 distances

✓ COMPLETE: 5 total distances

=== FIXED Statistics ===
total_shops_in_matrix: 6
pending_updates: 0
total_distances_stored: 15
expected_distances: 15
matrix_completeness: 100.0%

Distance Matrix:
['0.00', '18.68', '7.09', '18.48']
['18.68', '0.00', '31.69', '3.47']
['7.09', '31.69', '0.00', '30.03']
['18.48', '3.47', '30.03', '0.00']
Shop 1 → 2: 7.09 km, 8.75 min


In [1]:
%cd ..

c:\Users\HP\OneDrive - University of Moratuwa\Desktop\TMS-Project\TMS-Backend


In [2]:
from src.utils.master_utils import get_osrm_data


coords, distance, time = get_osrm_data((6.1234456,81.23243535), (6.3565777, 81.3454657))


In [3]:
coords

[[6.175183, 81.231847],
 [6.175658, 81.231812],
 [6.176242, 81.231812],
 [6.176518, 81.231763],
 [6.177925, 81.231843],
 [6.178346, 81.231832],
 [6.181333, 81.231358],
 [6.181677, 81.231267],
 [6.182169, 81.231231],
 [6.182303, 81.231248],
 [6.182579, 81.231292],
 [6.182831, 81.231362],
 [6.18305, 81.231512],
 [6.183067, 81.231704],
 [6.183015, 81.231994],
 [6.182803, 81.233173],
 [6.182575, 81.234066],
 [6.18236, 81.234613],
 [6.182281, 81.23502],
 [6.182233, 81.235421],
 [6.182236, 81.235902],
 [6.182337, 81.237554],
 [6.182984, 81.237423],
 [6.183311, 81.237303],
 [6.183753, 81.237147],
 [6.183859, 81.237116],
 [6.186183, 81.236481],
 [6.186683, 81.236321],
 [6.187044, 81.236195],
 [6.187361, 81.236078],
 [6.187493, 81.236072],
 [6.187683, 81.236101],
 [6.187957, 81.236306],
 [6.188102, 81.23648],
 [6.188204, 81.23672],
 [6.188384, 81.237008],
 [6.188565, 81.237182],
 [6.188784, 81.237241],
 [6.18907, 81.237261],
 [6.189266, 81.237351],
 [6.189368, 81.237481],
 [6.189411, 81.237659]

In [4]:
coords[::-1]

[[6.344807, 81.342425],
 [6.344759, 81.342083],
 [6.344215, 81.342108],
 [6.344244, 81.34265],
 [6.343773, 81.342749],
 [6.343709, 81.34214],
 [6.343594, 81.342154],
 [6.343447, 81.34213],
 [6.343072, 81.342017],
 [6.342957, 81.341947],
 [6.342634, 81.341867],
 [6.342525, 81.341856],
 [6.342409, 81.340854],
 [6.34229, 81.339009],
 [6.339691, 81.339158],
 [6.339372, 81.33483],
 [6.339368, 81.334776],
 [6.339185, 81.332157],
 [6.339137, 81.331138],
 [6.338981, 81.32954],
 [6.338837, 81.326909],
 [6.338801, 81.32668],
 [6.338598, 81.324229],
 [6.338582, 81.324105],
 [6.338582, 81.323349],
 [6.338544, 81.322308],
 [6.338486, 81.321402],
 [6.338335, 81.320663],
 [6.338241, 81.3202],
 [6.338188, 81.319937],
 [6.33814, 81.318612],
 [6.338203, 81.318194],
 [6.338384, 81.317609],
 [6.338822, 81.316595],
 [6.339024, 81.315823],
 [6.339062, 81.315517],
 [6.339065, 81.314822],
 [6.339073, 81.31333],
 [6.339077, 81.312883],
 [6.33904, 81.312604],
 [6.338982, 81.312447],
 [6.338962, 81.312297],
 [6.